# FAISS 비동기(Asynchronous)

본 튜토리얼은 FAISS DB 를 비동기로 활용하는 방법을 다룹니다.

더욱 자세한 사용법은 [Faiss 공식 문서](https://faiss.ai/)를 참조하세요.


faiss-gpu 또는 faiss-cpu 라이브러리를 설치합니다.

- CUDA 7.5 이상을 지원하는 GPU가 있는 경우 `faiss-gpu`를 설치합니다.
- GPU가 없거나 CPU에서 실행하려는 경우 `faiss-cpu`를 설치합니다.


In [1]:
# %pip install --upgrade --quiet  faiss-gpu # CUDA 7.5 이상을 지원하는 GPU용 설치
# 또는
# %pip install --upgrade --quiet  faiss-cpu # CPU용 설치

혹은 `dot_env` 라이브러리를 사용하여 `.env` 파일을 만들어 환경 변수로 설정할 수 있습니다.


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

추적을 위한 LangSmith 설정(필수는 아닙니다.)


In [ ]:
# LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# LANGCHAIN_API_KEY 환경 변수를 getpass.getpass() 함수를 통해 입력받은 값으로 설정합니다.
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN API KEY 입력"

- `TextLoader`를 사용하여 텍스트 데이터를 로드합니다.
- `CharacterTextSplitter`를 사용하여 로드된 문서를 1000자 단위로 분할하고, 분할된 문서 간에 중복되는 내용이 없도록 설정합니다.
- `OpenAIEmbeddings`를 사용하여 문서 임베딩을 생성합니다.
- `FAISS` 벡터 저장소를 초기화하고, 분할된 문서와 임베딩을 사용하여 벡터 인덱스를 구축합니다.

**참고**

- `AVX2`: 고도의 병렬 처리가 가능한 연산을 사용하는 벡터화 가능 알고리즘의 경우 `AVX2` 를 사용하면 CPU 성능이 향상되어 지연 시간이 줄어들며 처리량이 향상됩니다.
- 필요한 경우 `os.environ['FAISS_NO_AVX2'] = '1'` 코드 라인의 주석을 해제하여 FAISS에서 AVX2 최적화를 사용하지 않도록 설정할 수 있습니다.


샘플 데이터(`appendix-keywords.txt`) 일부 파일의 내용을 확인합니다.


In [4]:
# data/appendix-keywords.txt 파일 내용을 읽어서 file 변수에 저장합니다.
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

# file 변수에 저장된 내용을 출력합니다.
print(file[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


In [3]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader


# FAISS에서 AVX2 최적화를 사용하지 않으려면 다음 줄의 주석을 해제하세요.
# import os
#
# os.environ['FAISS_NO_AVX2'] = '1'

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = await FAISS.afrom_documents(docs, embeddings)

In [5]:
# 검색할 쿼리 설정
query = "임베딩(Embedding)이란 무엇인가요?"

# 쿼리와 유사한 문서 검색
docs = await db.asimilarity_search(query)

# 검색된 문서의 내용 출력
print(docs[0].page_content)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


## 점수에 기반한 유사도 검색

FAISS에는 몇 가지 특정 메서드가 있습니다.

그 중 하나는 `similarity_search_with_score`로, 문서뿐만 아니라 쿼리와 문서 간의 거리 점수도 반환할 수 있습니다.

반환되는 거리 점수는 L2 거리입니다.

따라서 점수가 낮을수록 더 좋은 결과입니다.


In [7]:
# 쿼리와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.
docs_and_scores = await db.asimilarity_search_with_score(query)

# 검색 결과 중 가장 유사도가 높은 문서와 점수를 가져옵니다.
docs_and_scores[0]

(Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'}),
 0.29650298)

`similarity_search_by_vector` 함수를 사용하면 주어진 임베딩 벡터와 유사한 문서를 검색할 수 있습니다.

이 함수는 문자열 대신 **임베딩 벡터** 를 매개변수로 받아들입니다.


In [8]:
# 검색할 쿼리 설정
query = "임베딩(Embedding)이란 무엇인가요?"
# 쿼리를 임베딩 벡터로 변환합니다.
embedding_vector = await embeddings.aembed_query(query)
# 임베딩 벡터를 사용하여 유사도 검색을 수행하고 문서와 점수를 반환합니다.
docs_and_scores = await db.asimilarity_search_by_vector(embedding_vector)

## 저장 및 로드

FAISS 인덱스를 저장하고 불러올 수도 있습니다.

이는 인덱스를 사용할 때마다 매번 다시 생성할 필요가 없어 유용합니다.


- `db.save_local("저장할 인덱스 이름")`를 사용하여 FAISS 인덱스를 로컬 디렉토리에 저장합니다.


In [11]:
# 로컬에 "MY_ASYNC_DB_INDEX"라는 이름으로 데이터베이스를 저장합니다.
DB_INDEX = "MY_ASYNC_DB_INDEX"
db.save_local(DB_INDEX)

- `FAISS.load_local("불러올 인덱스 이름", embeddings)`를 사용하여 저장된 FAISS 인덱스를 로드합니다.


In [13]:
# 로컬에 저장된 데이터베이스를 불러와 new_db 변수에 할당합니다.
new_db = FAISS.load_local(DB_INDEX, embeddings,
                          allow_dangerous_deserialization=True)

query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 비동기적으로 검색하여 docs 변수에 할당합니다.
docs = await new_db.asimilarity_search(query)

docs[0]  # 검색 결과 중 가장 유사한 문서를 반환합니다.

Document(page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken', metadata={'source': './data/appendix-keywords.txt'})

# 바이트 형태로 직렬화/역직렬화

FAISS 인덱스를 직렬화(Serializing)하고 역직렬화(De-Serializing)하는 데에는 `pickle` 함수를 사용하여 저장할 수 있습니다. 하지만, 만약 90MB 크기의 임베딩 모델(예: `sentence-transformers/all-MiniLM-L6-v2` 또는 다른 모델)을 사용한다면, 결과로 생성되는 pickle 파일의 크기는 90MB 이상이 될 것입니다. 이는 모델의 크기 또한 전체 크기에 포함되기 때문입니다.

따라서, 아래의 함수들을 사용하는 것이 좋습니다. 이 함수들은 **FAISS 인덱스만 직렬화** 하므로 크기가 훨씬 작아집니다.


In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# HuggingFaceEmbeddings 사용하여 임베딩을 생성합니다.
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, hf_embeddings)

/Users/teddy/miniconda3/envs/py-test/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
# FAISS 인덱스를 직렬화합니다.
serialized_db_index = db.serialize_to_bytes()

직렬화된 `serialized_db_index` 의 사이즈를 확인합니다.


In [16]:
import sys

# 사이즈 측정을 위한 함수를 정의합니다.


def get_size(path):
    size = sys.getsizeof(path)
    if size < 1024:
        return f"{size} bytes"
    elif size < pow(1024, 2):
        return f"{round(size/1024, 2)} KB"
    elif size < pow(1024, 3):
        return f"{round(size/(pow(1024,2)), 2)} MB"
    elif size < pow(1024, 4):
        return f"{round(size/(pow(1024,3)), 2)} GB"


# 직렬화된 FAISS 인덱스의 사이즈를 출력합니다.
get_size(serialized_db_index)

'60.81 KB'

이번에는 `deserialize_from_bytes` 함수로 역직렬화를 수행합니다.


In [18]:
# 직렬화된 인덱스를 로드합니다.
deserialized_db = FAISS.deserialize_from_bytes(
    embeddings=hf_embeddings,  # 직렬화 할 때의 임베딩과 동일하게 지정
    serialized=serialized_db_index,  # 직렬화된 인덱스
)

In [19]:
query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = deserialized_db.similarity_search(query)

# 문서 리스트의 첫 번째 문서를 가져옵니다.
docs[0]

Document(page_content='정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer', metadata={'source': './data/appendix-keywords.txt'})

## 병합(merge)

FAISS 벡터 저장소를 병합하는 것도 가능합니다.

두 개의 FAISS 벡터 저장소를 하나로 합칠 수 있습니다.


- `FAISS.afrom_texts` 메서드를 사용하여 두 개의 FAISS 데이터베이스 `db1`과 `db2`를 비동기적으로 생성합니다.
- `db1`은 `["foo"]` 텍스트 리스트로부터 생성되며, `embeddings`를 사용하여 벡터화됩니다.
- `db2`는 `["bar"]` 텍스트 리스트로부터 생성되며, 동일한 `embeddings`를 사용하여 벡터화됩니다.


In [20]:
# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# db1 생성
db1 = FAISS.from_texts(["LangChain DB 1의 내용"], embeddings)
# db2 생성
db2 = FAISS.from_texts(["DB 2 의 내용"], embeddings)

`db1` 의 내용 확인


In [21]:
# db1 벡터 데이터베이스의 내부 문서 저장소 딕셔너리에 접근합니다.
db1.docstore._dict

{'5ad3475d-3a95-469c-b868-8ab93b644708': Document(page_content='LangChain DB 1의 내용')}

`db2` 의 내용 확인


In [22]:
# db2 문서 저장소의 내부 딕셔너리에 접근합니다.
db2.docstore._dict

{'3439056f-c28f-4366-baa0-c79a949378c6': Document(page_content='DB 2 의 내용')}

`db1` 객체의 `merge_from` 메서드를 사용하여 `db2` 객체의 내용을 `db1`에 병합합니다.

- `merge_from` 메서드는 `db2`의 데이터를 `db1`에 병합하는 기능을 수행합니다.
- 병합 작업 후에는 `db1`에 `db2`의 데이터가 포함됩니다.


In [23]:
db1.merge_from(db2)  # db1에 db2를 병합합니다.

병합된 결과를 확인합니다.


In [24]:
# db1 문서 저장소의 내부 딕셔너리에 접근합니다.
db1.docstore._dict

{'5ad3475d-3a95-469c-b868-8ab93b644708': Document(page_content='LangChain DB 1의 내용'),
 '3439056f-c28f-4366-baa0-c79a949378c6': Document(page_content='DB 2 의 내용')}

## 필터링

FAISS vectorstore는 필터링 기능도 지원할 수 있습니다.

FAISS는 기본적으로 필터링을 지원하지 않기 때문에, 이를 수동으로 처리해야 합니다.

**방식**

1. 이는 먼저 `k`보다 더 많은 결과를 가져온 다음 필터링하는 방식으로 이루어집니다.

2. 이 필터는 메타데이터 dict를 입력으로 받아 bool을 반환하는 호출 가능한 함수이거나, 누락된 각 키는 무시되고 존재하는 각 키는 값 목록에 있어야 하는 메타데이터 dict입니다.

또한 검색 메서드를 호출할 때 `fetch_k` 매개변수를 설정하여 필터링 전에 가져올 문서 수를 지정할 수 있습니다.


In [ ]:
from langchain_core.documents import Document

list_of_documents = [
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="foo", metadata=dict(page=1)),
    # 페이지 내용이 "bar"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="bar", metadata=dict(page=1)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="foo", metadata=dict(page=2)),
    # 페이지 내용이 "barbar"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="barbar", metadata=dict(page=2)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="foo", metadata=dict(page=3)),
    # 페이지 내용이 "bar burr"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="bar burr", metadata=dict(page=3)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="foo", metadata=dict(page=4)),
    # 페이지 내용이 "bar bruh"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
# 문서 리스트와 임베딩을 사용하여 FAISS 데이터베이스 생성
db = FAISS.from_documents(list_of_documents, embeddings)

# "foo"와 유사한 문서를 검색하고 점수와 함께 결과 반환
results_with_scores = db.similarity_search_with_score("foo")

for doc, score in results_with_scores:  # 검색 결과를 반복하면서
    # 각 문서의 내용, 메타데이터, 점수를 출력
    print(
        f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

In [28]:
from langchain_core.documents import Document

list_of_documents = [
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="foo", metadata=dict(page=1)),
    # 페이지 내용이 "bar"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="bar", metadata=dict(page=1)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="foo", metadata=dict(page=2)),
    # 페이지 내용이 "barbar"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="barbar", metadata=dict(page=2)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="foo", metadata=dict(page=3)),
    # 페이지 내용이 "bar burr"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="bar burr", metadata=dict(page=3)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="foo", metadata=dict(page=4)),
    # 페이지 내용이 "bar bruh"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
# 문서 리스트와 임베딩을 사용하여 FAISS 데이터베이스 생성
db = FAISS.from_documents(list_of_documents, embeddings)
results_with_scores = db.asimilarity_search_with_score(
    "foo"
)  # "foo"와 유사한 문서를 검색하고 점수와 함께 결과 반환

이제 동일한 쿼리 호출을 수행하지만, `page = 1`인 경우만 필터링합니다.


In [ ]:
# 유사도 검색을 수행하고 결과를 점수와 함께 가져옵니다. 필터로 page가 1인 문서만 검색합니다.
results_with_scores = await db.asimilarity_search_with_score("foo", filter=dict(page=1))
for (
    doc,
    score,
) in results_with_scores:  # 검색 결과를 반복하면서 각 문서와 점수를 가져옵니다.
    # 문서의 내용, 메타데이터, 점수를 출력합니다.
    print(
        f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

## Max Marginal Relevance (MMR)

`MMR(Maximal Marginal Relevance)` 방식은 쿼리에 대한 관련 항목을 검색할 때 중복을 피하는 방법 중 하나입니다. 단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 검색된 항목들 사이에 **관련성과 다양성 사이의 균형을 보장**합니다. 이는 **자주 발생할 수 있는, 매우 유사한 항목들만이 검색되는 상황을 방지** 하는 데에 유용합니다.

예를 들어, 특정 주제에 대해 정보를 찾고 있다고 가정해봅시다. 가장 관련성 높은 문서만을 반환하는 시스템은 비슷비슷한 정보를 담은 문서들을 제공할 수 있습니다. 하지만, MMR 방식을 사용하면, 검색된 문서들이 해당 주제에 대해 서로 다른 관점이나 새로운 정보를 제공하도록 합니다. 이로써 사용자는 주제에 대해 보다 폭넓은 이해를 할 수 있게 됩니다.

MMR은 두 가지 주요 요소, 즉 쿼리에 대한 문서의 관련성과 **이미 선택된 문서들과의 차별성을 동시에 고려** 합니다.

1. 첫째로, 쿼리와의 관련성이 높은 문서를 찾는 것이 중요합니다.
2. 둘째로, 이미 선택된 문서들과는 다른 새로운 정보나 관점을 제공하는 문서를 찾는 것입니다.

이 두 가지 요소의 균형을 맞추는 것이 MMR의 핵심입니다.


In [29]:
# "foo"를 검색어로 사용하여 최대 한계 관련성 검색을 수행하고, 필터로 page가 1인 문서만 검색합니다.
results = await db.amax_marginal_relevance_search("foo", filter=dict(page=1))
for doc in results:  # 검색 결과를 반복합니다.
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}
Content: bar, Metadata: {'page': 1}


`similarity_search` 함수를 호출할 때 `fetch_k` 매개변수를 설정하는 방법에 대한 예시입니다.

일반적으로 `fetch_k` 매개변수는 `k` 매개변수보다 훨씬 큰 값으로 설정하는 것이 좋습니다. 그 이유는 `fetch_k` 매개변수가 필터링 전에 가져올 문서의 수를 나타내기 때문입니다.

만약 `fetch_k`를 낮은 값으로 설정하면, 필터링할 문서가 충분하지 않을 수 있습니다.


In [30]:
# "foo"와 유사한 문서를 검색하고, 'page' 메타데이터가 1인 문서만 필터링하여 가장 유사한 1개의 문서를 반환하되, 4개의 문서를 가져옵니다.
results = await db.asimilarity_search("foo", filter=dict(page=1), k=1, fetch_k=4)
for doc in results:  # 검색 결과를 반복하면서 각 문서에 대해 다음을 수행합니다.
    # 문서의 내용과 메타데이터를 출력합니다.
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: foo, Metadata: {'page': 1}


## 문서 삭제

벡터 저장소에 저장된 문서를 삭제할 수도 있습니다. 이때 삭제할 문서의 ID는 DB 에 저장된 ID와 일치해야 합니다.


`db.delete()` 메서드를 사용하여 문서를 삭제합니다.


In [31]:
db.index_to_docstore_id[0]

'660aafd1-5952-4caf-b55d-e7cc3db10712'

In [32]:
# 인덱스 0에 해당하는 문서 저장소 ID를 사용하여 데이터베이스에서 문서를 삭제합니다.
db.delete([db.index_to_docstore_id[0]])

True